In [ ]:
import os
os.environ['PYTHONWARNINGS'] = 'ignore::ImportWarning'

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=ImportWarning)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Install necessary libraries
!pip install flask flask-ngrok
!pip install numpy==1.26.4 --force-reinstall
import numpy as np
import pandas as pd

  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.0 which is incompatible.


In [ ]:
!pip install scipy==1.13.0
!pip install mne==0.22.0

In [ ]:
import os
import time
import pickle

from scipy import signal
from scipy.signal import welch
from scipy.integrate import simps
from scipy.stats import f_oneway

#!pip install scikit-learn==0.20.3
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import model_selection
from sklearn.metrics import classification_report, confusion_matrix
import itertools
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve

import mne
from mne.preprocessing import (ICA, create_eog_epochs, create_ecg_epochs, corrmap)
from mne.time_frequency import psd_welch
from mne.decoding import cross_val_multiscore

!pip install fooof
from fooof import FOOOFGroup
from fooof.bands import Bands
from fooof.analysis import get_band_peak_fg
from fooof.plts.spectra import plot_spectrum

import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
%matplotlib inline

In [ ]:
from flask import Flask, request, render_template, jsonify
from flask_ngrok import run_with_ngrok  # Use this for Colab or remote environments
import os
import pickle
import numpy as np
import mne
from mne.preprocessing import ICA
from scipy.stats import pearsonr
from itertools import chain
import matplotlib.pyplot as plt
from joblib import load

In [ ]:
from joblib import load

In [ ]:
# Load pre-trained models
valence_model = load("/content/drive/MyDrive/flask deployment/MLPClassifier_valence(alpha=1e-05, hidden_layer_sizes=(92, 46), learning_rate_init=0.09,              max_iter=1000, n_iter_no_change=80).pkl")
arousal_model = load("/content/drive/MyDrive/flask deployment/MLPClassifier_arousal(alpha=1e-05, hidden_layer_sizes=(53, 50), learning_rate_init=0.09,              max_iter=1000, n_iter_no_change=80).pkl")
dominance_model = load("/content/drive/MyDrive/flask deployment/MLPClassifier_dominance(activation='tanh', alpha=1e-05, hidden_layer_sizes=(58, 33),              learning_rate_init=0.09, max_iter=1000, n_iter_no_change=80,              solver='sgd').pkl")
liking_model = load("/content/drive/MyDrive/flask deployment/MLPClassifier_Liking(alpha=1e-05, hidden_layer_sizes=(53, 25), learning_rate_init=0.09,              max_iter=1000, n_iter_no_change=80).pkl")

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LabelBinarizer from version 1.5.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator MLPClassifier from version 1.5.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LabelBinarizer from version 1.5.2 when using version 1.6.1. This might lead to breaking code or i

In [ ]:
def get_feature(data):
    channel_no = [0, 2, 16, 19]  # Specify channels to use
    feature_matrix = []

    # Process each channel specified in channel_no
    for ith_channel in channel_no:
        signal = data[ith_channel, :]  # Extract the signal for the channel

        # Compute PSD
        psd, freqs = plt.psd(signal, Fs=128, NFFT=256)

        # Extract frequency band powers
        theta_mean = np.mean(psd[np.logical_and(freqs >= 4, freqs <= 7)])
        alpha_mean = np.mean(psd[np.logical_and(freqs >= 8, freqs <= 13)])
        beta_mean = np.mean(psd[np.logical_and(freqs >= 13, freqs <= 30)])
        gamma_mean = np.mean(psd[np.logical_and(freqs >= 30, freqs <= 40)])

        feature_matrix.append([theta_mean, alpha_mean, beta_mean, gamma_mean])

    # Flatten the feature matrix for all selected channels
    return np.array(list(chain.from_iterable(feature_matrix)))
# Signal processing and feature extraction functions
def SignalPreProcess(eeg_rawdata):
    print("in signal preprocess")
    N_C = 20  # Number of ICA components
    droping_components = 'one'  # Drop one or two components
    ch_names = [
        "Fp1", "AF3", "F3", "F7", "FC5", "FC1", "C3", "T7", "CP5", "CP1", "P3", "P7", "PO3", "O1", "Oz",
        "Pz", "Fp2", "AF4", "Fz", "F4", "F8", "FC6", "FC2", "Cz", "C4", "T8", "CP6", "CP2", "P4", "P8",
        "PO4", "O2"
    ]
    info = mne.create_info(ch_names=ch_names, ch_types=['eeg'] * 32, sfreq=128, verbose=False)
    raw_data = mne.io.RawArray(eeg_rawdata, info, verbose=False)
    raw_data.load_data().filter(l_freq=4, h_freq=48, method='fir', verbose=False)
    print("before ica")
    ica = ICA(n_components=N_C, random_state=97, verbose=False)
    ica.fit(raw_data)
    ica_sources = ica.get_sources(raw_data).get_data()
    eog_signal = raw_data.copy().pick_channels(['Fp1']).get_data()[0]
    print("before correlations")
    correlations = np.array([pearsonr(component, eog_signal)[0] for component in ica_sources])
    if droping_components == 'one':
        ica.exclude = [np.argmax(np.abs(correlations))]
    else:
        top_two_indices = np.argsort(np.abs(correlations))[-2:]
        ica.exclude = list(top_two_indices)

    ica.apply(raw_data, verbose=False)
    raw_data.set_eeg_reference('average', ch_type='eeg')
    return np.array(raw_data.get_data())

def signal_pro(input_data):
    print("in signal pro")
    input_data = SignalPreProcess(input_data.copy())
    return input_data

In [ ]:
import joblib
def facial_prediction(X):

  path_valence = "/content/drive/MyDrive/for_abrar_frontend_testing/fyp3_valence.pkl"
  path_arousal = "/content/drive/MyDrive/for_abrar_frontend_testing/fyp3_arousal.pkl"
  path_dominance = "/content/drive/MyDrive/for_abrar_frontend_testing/fyp3_dominance.pkl"
  path_liking = "/content/drive/MyDrive/for_abrar_frontend_testing/fyp3_liking.pkl"

  # Load the models
  model_valence = joblib.load(path_valence)
  model_arousal = joblib.load(path_arousal)
  model_dominance = joblib.load(path_dominance)
  model_liking = joblib.load(path_liking)

  # Make predictions on the test data
  y_pred_valence = model_valence.predict(X)
  y_pred_arousal = model_arousal.predict(X)
  y_pred_dominance = model_dominance.predict(X)
  y_pred_liking = model_liking.predict(X)

  binary_valence = (y_pred_valence > 0.5).astype(int)
  binary_arousal = (y_pred_arousal > 0.5).astype(int)
  binary_dominance = (y_pred_dominance > 0.5).astype(int)
  binary_liking = (y_pred_liking > 0.5).astype(int)

  return binary_valence,binary_arousal,binary_dominance,binary_liking

In [ ]:
def facial_preprocess(main_df):
    print(main_df.info())
    print("Missing values before cleanup:\n", main_df.isnull().sum())

    main_df = main_df.dropna()

    for col in main_df.columns:
        if main_df[col].dtype == 'float64':
            main_df[col] = main_df[col].fillna(main_df[col].mean())
        elif main_df[col].dtype == 'int64':
            main_df[col] = main_df[col].fillna(main_df[col].median())

    print("Missing values after cleanup:\n", main_df.isnull().sum())

    if 'frame' in main_df.columns:
        main_df['frame'] = main_df['frame'].astype(int)

    # Extract features
    X = main_df.iloc[:, 5:-4].values  # shape: (n_frames, n_features)

    # Use average or first frame
    X_avg = X.mean(axis=0).reshape(1, -1)  # shape: (1, n_features)
    print("Input shape to facial_prediction:", X_avg.shape)

    valence, arousal, dominance, liking = facial_prediction(X_avg)

    return valence, arousal, dominance, liking

In [ ]:
def get_group_name(valence, arousal, dominance, liking):
    """
    Returns the group name based on binary values for valence, arousal, dominance, and liking.
    """
    valence_label = "Hv" if valence == 1 else "Lv"
    arousal_label = "Ha" if arousal == 1 else "La"
    dominance_label = "Hd" if dominance == 1 else "Ld"
    liking_label = "Hl" if liking == 1 else "Ll"

    return f"{valence_label}{arousal_label}{dominance_label}{liking_label}"

In [ ]:
def get_emotions_from_group(group_name):
    """
    Returns the emotional description for a given group name.

    Args:
        group_name (str): The group name in the format "HvLaHdLl".

    Returns:
        str: Description of the emotions associated with the group.
    """
    # Mapping of group names to emotions
    group_emotions = {
        "LvLaLdLl": "Apathetic, Disconnected",
        "LvLaLdHl": "Indifferent but Favorable",
        "LvLaHdLl": "Submissive, Defeated",
        "LvLaHdHl": "Sympathetic, Mildly Caring",
        "LvHaLdLl": "Agitated, Stressed",
        "LvHaLdHl": "Frustrated but Favorable",
        "LvHaHdLl": "Angry, Overwhelmed",
        "LvHaHdHl": "Determined but Critical",
        "HvLaLdLl": "Calm but Detached",
        "HvLaLdHl": "Relaxed and Affectionate",
        "HvLaHdLl": "Confident but Reserved",
        "HvLaHdHl": "Content, Grateful",
        "HvHaLdLl": "Excited but Unconnected",
        "HvHaLdHl": "Energetic and Loving",
        "HvHaHdLl": "Passionate, Assertive",
        "HvHaHdHl": "Joyful, Enthusiastic, Empowered"
    }

    # Fetch and return the emotion description
    emotion_description = group_emotions.get(group_name, "Unknown Group Name")
    return emotion_description

In [ ]:
!pip install pyngrok

/usr/local/lib/python3.11/dist-packages/google/colab/_pip.py:86: ResourceWarning: unclosed file <_io.TextIOWrapper name='/usr/local/lib/python3.11/dist-packages/pyngrok-7.2.8.dist-info/top_level.txt' mode='r' encoding='utf-8'>
  for line in open(toplevel):


In [ ]:
from flask import Flask, request, jsonify, render_template_string
import io
import os
import shutil
import pickle
import numpy as np
from pyngrok import ngrok

In [ ]:
import os
import pickle
from flask import Flask, request, jsonify, render_template_string
import numpy as np
from pyngrok import ngrok
import pandas as pd

# Flask app setup
app = Flask(__name__)

# Set up ngrok with your auth token
NGROK_AUTH_TOKEN = "2x32v40atG66GdDAzLUPj4xcDqb_5gcCsp6eKowtceN4MkTDY"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# HTML content embedded directly in the Flask app
html_content = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>InsightsIM - EEG & Facial Data Uploader</title>
    <style>
        /* Global styles */
        * {
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }

        body {
            font-family: 'Roboto', Arial, sans-serif;
            background: linear-gradient(to bottom, #ffffff, #f7f9fc);
            color: #2c3e50;
            line-height: 1.6;
            display: flex;
            flex-direction: column;
            min-height: 100vh;
        }

        header {
            background: linear-gradient(90deg, #4caf50, #1976d2);
            color: #ffffff;
            padding: 20px 40px;
            box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
            display: flex;
            align-items: center;
            justify-content: space-between;
        }

        header h1 {
            font-size: 28px;
            font-weight: bold;
        }

        header .logo {
            font-size: 18px;
            font-weight: 300;
            opacity: 0.8;
        }

        .main-container {
            flex: 1;
            display: flex;
            max-width: 1200px;
            margin: 40px auto;
            gap: 30px;
            padding: 0 20px;
        }

        .sidebar {
            width: 280px;
            background: #1976d2;
            color: #ffffff;
            border-radius: 10px;
            padding: 20px;
            box-shadow: 0 2px 10px rgba(0, 0, 0, 0.2);
        }

        .sidebar h2 {
            font-size: 22px;
            margin-bottom: 15px;
            border-bottom: 2px solid #ffffff;
            padding-bottom: 5px;
        }

        .sidebar p {
            font-size: 16px;
            opacity: 0.9;
            margin-bottom: 15px;
        }

        .content {
            flex: 1;
            background: #ffffff;
            border-radius: 10px;
            box-shadow: 0 2px 10px rgba(0, 0, 0, 0.1);
            padding: 20px;
            display: flex;
            flex-direction: column;
            gap: 20px;
        }

        .form-container {
            background: #f9f9f9;
            border: 1px solid #e0e0e0;
            padding: 20px;
            border-radius: 8px;
            box-shadow: 0 2px 5px rgba(0, 0, 0, 0.1);
        }

        .form-container h1 {
            font-size: 20px;
            margin-bottom: 15px;
        }

        form {
            display: flex;
            flex-direction: column;
            gap: 15px;
        }

        form label {
            font-weight: bold;
        }

        form input[type="file"] {
            padding: 10px;
            border: 1px solid #ccc;
            border-radius: 5px;
            background: #fdfdfd;
        }

        form button {
            background-color: #4caf50;
            color: white;
            border: none;
            padding: 12px;
            font-size: 16px;
            font-weight: bold;
            border-radius: 5px;
            cursor: pointer;
            transition: all 0.3s;
        }

        form button:hover {
            background-color: #388e3c;
        }

        .tabs {
            display: flex;
            gap: 10px;
            margin-bottom: 20px;
            border-bottom: 2px solid #ddd;
        }

        .tabs button {
            background: none;
            border: none;
            padding: 10px 20px;
            cursor: pointer;
            font-size: 16px;
            font-weight: bold;
            color: #2c3e50;
            border-radius: 5px 5px 0 0;
            transition: all 0.3s;
        }

        .tabs button.active {
            background-color: #4caf50;
            color: #ffffff;
            border: 1px solid #4caf50;
            border-bottom: none;
        }

        .tab-content {
            display: none;
            background: #ffffff;
            border-radius: 8px;
            padding: 20px;
            border: 1px solid #e0e0e0;
            box-shadow: 0 2px 5px rgba(0, 0, 0, 0.1);
        }

        .tab-content.active {
            display: block;
        }

        footer {
            background: #2c3e50;
            color: #ffffff;
            text-align: center;
            padding: 10px;
            margin-top: auto Ağustos 2024
            box-shadow: 0 -2px 6px rgba(0, 0, 0, 0.1);
        }

        footer p {
            font-size: 14px;
        }

        footer a {
            color: #4caf50;
            text-decoration: none;
            font-weight: bold;
        }
    </style>
</head>
<body>

    <!-- Header -->
    <header>
        <h1>InsightsIM</h1>
        <span class="logo">EEG & Facial Data Analysis Platform</span>
    </header>

    <!-- Main Layout -->
    <div class="main-container">

        <!-- Sidebar -->
        <div class="sidebar">
            <h2>About InsightsIM</h2>
            <p>Effortlessly upload, analyze, and visualize EEG and facial data with our intuitive platform.</p>
            <p>Navigate through tabs to view predictions, group data, and related emotions.</p>
        </div>

        <!-- Main Content -->
        <div class="content">

            <!-- File Upload Form -->
            <div class="form-container">
                <h1>Upload EEG and Facial Data Files</h1>
                <form id="upload-form" action="/upload" method="POST" enctype="multipart/form-data">
                    <label for="dat_file">Select EEG .dat file:</label>
                    <input type="file" name="dat_file" id="dat_file" accept=".dat" required>
                    <label for="csv_file">Select Facial .csv file:</label>
                    <input type="file" name="csv_file" id="csv_file" accept=".csv" required>
                    <button type="submit" id="submit-button">Upload Files</button>
                </form>
            </div>

            <!-- Tabs -->
            <div class="tabs">
                <button class="tab-button active" data-tab="results">Model Results</button>
                <button class="tab-button" data-tab="groups">Groups</button>
                <button class="tab-button" data-tab="emotions">Emotions</button>
            </div>

            <!-- Tab Content -->
            <div id="results" class="tab-content active">
                <h2>Model Results</h2>
                <pre id="results-text">Results will be displayed here...</pre>
            </div>

            <div id="groups" class="tab-content">
                <h2>Groups</h2>
                <pre id="groups-text">Group details will be displayed here...</pre>
            </div>

            <div id="emotions" class="tab-content">
                <h2>Emotions</h2>
                <pre id="emotions-text">Emotion data will be displayed here...</pre>
            </div>

        </div>
    </div>

    <!-- Footer -->
    <footer>
        <p>© 2024 InsightsIM. Built with ❤️ for EEG and facial analysis. <a href="#">Learn More</a></p>
    </footer>

    <script>
        const form = document.getElementById('upload-form');
        const resultsText = document.getElementById('results-text');
        const groupsText = document.getElementById('groups-text');
        const emotionsText = document.getElementById('emotions-text');

        const tabButtons = document.querySelectorAll('.tab-button');
        const tabContents = document.querySelectorAll('.tab-content');

        tabButtons.forEach(button => {
            button.addEventListener('click', () => {
                tabButtons.forEach(btn => btn.classList.remove('active'));
                tabContents.forEach(content => content.classList.remove('active'));

                button.classList.add('active');
                document.getElementById(button.dataset.tab).classList.add('active');
            });
        });

        form.onsubmit = async (event) => {
            event.preventDefault();

            const formData = new FormData(form);
            resultsText.textContent = "Processing...";
            groupsText.textContent = "Waiting for group data...";
            emotionsText.textContent = "Waiting for emotion data...";

            try {
                const response = await fetch("/upload", {
                    method: 'POST',
                    body: formData
                });

                if (!response.ok) {
                    throw new Error(`Server responded with status ${response.status}`);
                }

                const result = await response.json();

                if (result.error) {
                    resultsText.textContent = `Error: ${result.error}`;
                } else {
                    resultsText.textContent = result.modelResults || "No model results available.";
                    groupsText.textContent = result.groups || "No group data available.";
                    emotionsText.textContent = result.emotions || "No emotion data available.";
                }

            } catch (error) {
                resultsText.textContent = `Error: ${error.message || "An unexpected error occurred."}`;
            }
        };
    </script>
</body>
</html>
"""

@app.route("/")
def home():
    return render_template_string(html_content)

@app.route("/upload", methods=["POST"])
def upload_files():
    print("in upload_files")
    try:
        print("enter try")
        #if 'dat_file' not in request.files or 'csv_file' not in request.files:
            #return jsonify({"error": "Missing EEG or facial data file."}), 400
        print("request files")
        dat_file = request.files['dat_file']
        csv_file = request.files['csv_file']
        print("check filenames")
        #if dat_file.filename == '' or csv_file.filename == '':
            #return jsonify({"error": "No file selected."}), 400
        print("get files and send to process")
        output = process_files(dat_file, csv_file)
        print("get output in upload_files")
        print(output["predictions"],output["groups"],output["emotions"])
        response = {
            "modelResults": output["predictions"],
            "groups": output["groups"],
            "emotions": output["emotions"]
        }
        print("response sent to front end")
        return jsonify(response)

    except Exception as e:
        return jsonify({"error": f"Error during file processing: {str(e)}"}), 500

def process_files(dat_file, csv_file):
    try:
        # Process EEG data
        deap_dataset = pickle.load(dat_file.stream, encoding='latin1')
        print("EEG file read")
        # Extract EEG data
        data = np.array(deap_dataset['data'])
        data = data[0:40, 0:32, 384:8064]
        data_32_channels = data[0][:32]
        print("EEG data extracted")
        # Process EEG data
        processed_eeg_data = signal_pro(data_32_channels)
        print("EEG data processed")
        eeg_feature = get_feature(processed_eeg_data)
        print("EEG features extracted")
        eeg_features = eeg_feature.reshape(1, -1)
        print("EEG features reshaped")

        # EEG Predictions
        eeg_valence_prediction = valence_model.predict(eeg_features)
        eeg_arousal_prediction = arousal_model.predict(eeg_features)
        eeg_dominance_prediction = dominance_model.predict(eeg_features)
        eeg_liking_prediction = liking_model.predict(eeg_features)
        print("EEG predictions obtained")
        # EEG Group and Emotions
        eeg_group_name = get_group_name(eeg_valence_prediction, eeg_arousal_prediction, eeg_dominance_prediction, eeg_liking_prediction)
        eeg_emotions = get_emotions_from_group(eeg_group_name)
        print("EEG group and emotions processed")

        # Process facial data
        facial_data = pd.read_csv(csv_file.stream)
        print("Facial CSV file read")
        facial_valence_prediction,facial_arousal_prediction,facial_dominance_prediction,facial_liking_prediction = facial_preprocess(facial_data)
        print("Facial predictions obtained")
        print(facial_valence_prediction,facial_arousal_prediction,facial_dominance_prediction,facial_liking_prediction)
        # Facial Group and Emotions
        facial_group_name = get_group_name(facial_valence_prediction, facial_arousal_prediction, facial_dominance_prediction, facial_liking_prediction)
        facial_emotions = get_emotions_from_group(facial_group_name)
        print("Facial group and emotions processed")

        # Prepare structured output
        return {
            "predictions": (
                f"EEG Predictions:\n"
                f"  Valence: {eeg_valence_prediction[0]}\n"
                f"  Arousal: {eeg_arousal_prediction[0]}\n"
                f"  Dominance: {eeg_dominance_prediction[0]}\n"
                f"  Liking: {eeg_liking_prediction[0]}\n"
                f"\nFacial Predictions:\n"
                f"  Valence: {facial_valence_prediction[0]}\n"
                f"  Arousal: {facial_arousal_prediction[0]}\n"
                f"  Dominance: {facial_dominance_prediction[0]}\n"
                f"  Liking: {facial_liking_prediction[0]}"
            ),
            "groups": (
                f"EEG Group: {eeg_group_name}\n"
                f"Facial Group: {facial_group_name}"
            ),
            "emotions": (
                f"EEG Emotions: {eeg_emotions}\n"
                f"Facial Emotions: {facial_emotions}"
            )
        }

    except Exception as e:
        return {"error": f"Error during file processing: {str(e)}"}

if __name__ == "__main__":
    # Start ngrok tunnel
    public_url = ngrok.connect(5000)
    print(f"ngrok tunnel available at: {public_url}")

    app.run(port=5000)

ngrok tunnel available at: NgrokTunnel: "https://b4e4-34-75-145-246.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [18/May/2025 19:11:13] "GET / HTTP/1.1" 200 -
sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7d7e1095f230>
INFO:werkzeug:127.0.0.1 - - [18/May/2025 19:11:14] "GET /favicon.ico HTTP/1.1" 404 -
sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7d7e1095f310>


in upload_files
enter try
request files
check filenames
get files and send to process
EEG file read
EEG data extracted
in signal pro
in signal preprocess
before ica


/usr/local/lib/python3.11/dist-packages/mne/filter.py:1009: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  logger.info('Setting up band-pass filter from %0.2g - %0.2g Hz'
/usr/local/lib/python3.11/dist-packages/mne/filter.py:1009: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  logger.info('Setting up band-pass filter from %0.2g - %0.2g Hz'
/usr/local/lib/python3.11/dist-packages/mne/filter.py:1702: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  l_freq = cast(l_freq)
/

before correlations
EEG data processed
EEG features extracted
EEG features reshaped
EEG predictions obtained
EEG group and emotions processed
Facial CSV file read
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Columns: 718 entries, frame to Liking
dtypes: float64(715), int64(3)
memory usage: 16.4 MB
None
Missing values before cleanup:
 frame          0
 face_id       0
 timestamp     0
 confidence    0
 success       0
              ..
 AU45_c        0
Valence        0
Arousal        0
Dominance      0
Liking         0
Length: 718, dtype: int64
Missing values after cleanup:
 frame          0
 face_id       0
 timestamp     0
 confidence    0
 success       0
              ..
 AU45_c        0
Valence        0
Arousal        0
Dominance      0
Liking         0
Length: 718, dtype: int64
Input shape to facial_prediction: (1, 709)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step


...

/usr/local/lib/python3.11/dist-packages/tensorflow/python/framework/ops.py:315: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float(self._numpy())
/usr/local/lib/python3.11/dist-packages/tensorflow/python/framework/ops.py:315: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float(self._numpy())


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step


INFO:werkzeug:127.0.0.1 - - [18/May/2025 19:14:02] "POST /upload HTTP/1.1" 200 -


Facial predictions obtained
[[0]] [[1]] [[1]] [[1]]
Facial group and emotions processed
get output in upload_files
EEG Predictions:
  Valence: 1
  Arousal: 0
  Dominance: 0
  Liking: 1

Facial Predictions:
  Valence: [0]
  Arousal: [1]
  Dominance: [1]
  Liking: [1] EEG Group: HvLaLdHl
Facial Group: LvHaHdHl EEG Emotions: Relaxed and Affectionate
Facial Emotions: Determined but Critical
response sent to front end


sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7d7e1095f230>
